In [ ]:
import json
from pathlib import Path
import shutil
from matplotlib import pyplot as plt
import numpy as np
from tqdm.auto import tqdm


gazecapture = Path().cwd() / "gazecapture"

axis_type = "Cam"
gazecapture_dirs = sorted(gazecapture.iterdir())
xmax_mean,xmin_mean = 0, 0
ymax_mean,ymin_mean = 0, 0
for dir in gazecapture_dirs:
	dotinfos = json.load(open(dir / "dotInfo.json", "r"))
	xs = np.array(dotinfos[f"X{axis_type}"])
	ys = np.array(dotinfos[f"Y{axis_type}"])
	xmax = np.max(xs)
	xmin = np.min(xs)
	ymax = np.max(ys)
	ymin = np.min(ys)
	xmax_mean += xmax
	xmin_mean += xmin
	ymax_mean += ymax
	ymin_mean += ymin
xmax_mean /= len(gazecapture_dirs)
xmin_mean /= len(gazecapture_dirs)
ymax_mean /= len(gazecapture_dirs)
ymin_mean /= len(gazecapture_dirs)

# print(f"X{axis_type} range: {xmin_mean} to {xmax_mean}")
# print(f"Y{axis_type} range: {ymin_mean} to {ymax_mean}")
	
zero = 0
one = 0
destination = Path("dataset")
destination.mkdir(exist_ok=True)
(destination / "0").mkdir(exist_ok=True)
(destination / "1").mkdir(exist_ok=True)

cnt = 0
for dir in tqdm(gazecapture_dirs):
	dotinfos = json.load(open(dir / "dotInfo.json", "r"))
	xs = np.array(dotinfos[f"X{axis_type}"])
	ys = np.array(dotinfos[f"Y{axis_type}"])
	
	frames_dir = sorted((dir / "frames").glob("*.jpg"))
	for x, y, frame_dir in zip(xs, ys, frames_dir):
		cnt += 1
		if x > xmax_mean*0.9 or x < xmin_mean*0.9 or y > ymax_mean or y < ymin_mean*0.9:
			zero+=1
			shutil.copy(frame_dir, destination / "0" / "{:08d}.jpg".format(cnt))
		if x < xmax_mean*0.4 and x > xmin_mean*0.4 and y < ymax_mean*0.4 and y > ymin_mean*0.4:
			one+=1
			shutil.copy(frame_dir, destination / "1" / "{:08d}.jpg".format(cnt))

# print(f"Zero: {zero}, One: {one}")

100%|██████████| 100/100 [00:38<00:00,  2.60it/s]

Zero: 35132, One: 51197
